##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# BigBiGAN으로 이미지 생성하기


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/bigbigan_with_tf_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/hub/tutorials/bigbigan_with_tf_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/hub/tutorials/bigbigan_with_tf_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/hub/tutorials/bigbigan_with_tf_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
  <td><a href="https://tfhub.dev/s?q=experts%2Fbert"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF Hub 모델보기</a></td>
</table>

이 노트북은 [TF Hub](https://tfhub.dev/s?publisher=deepmind&q=bigbigan)에서 사용할 수 있는 *BigBiGAN* 모델의 데모입니다.

BigBiGAN는 감독되지 않은 표현 학습에 사용할 수 있는 *인코더* 모듈을 추가하여 표준 (Big)GAN을 확장합니다. 대략적으로 말해서, 인코더는 `x`의 실제 데이터가 주어졌을 때 잠재 `z`를 예측하여 생성기를 반전시킵니다. 이러한 모델에 대한 자세한 내용은 [arXiv에 관한 BigBiGAN 논문](https://arxiv.org/abs/1907.02544)[1]을 참조하세요.

런타임에 연결한 후 다음 안내에 따라 시작합니다.

1. (선택 사항) 다른 인코더 아키텍처에 대한 BigBiGAN 생성기를 로드하려면 아래 첫 번째 코드 셀에서 선택한 **`module_path`**를 업데이트합니다.
2. **Runtime &gt; Run all**을 클릭하여 각 셀을 순서대로 실행합니다. 나중에 BigBiGAN 샘플과 재구성의 시각화를 포함한 출력이 아래에 자동으로 나타납니다.

참고: 문제가 발생하는 경우, **Runtime &gt; Restart and run all...**을 클릭하여 런타임을 다시 시작하고 모든 셀을 처음부터 다시 실행하면 도움이 될 수 있습니다.

[1] Jeff Donahue and Karen Simonyan. [Large Scale Adversarial Representation Learning](https://arxiv.org/abs/1907.02544). *arxiv:1907.02544*, 2019.

먼저 모듈 경로를 설정합니다. 기본적으로 **`https://tfhub.dev/deepmind/bigbigan-resnet50/1`**에서 더 작은 ResNet-50 기반 인코더로 BigBiGAN 모델을 로드합니다. 최상의 표현 학습 결과를 얻기 위해 사용되는 더 큰 RevNet-50-x4 기반 모델을 로드하려면 활성 **`module_path`** 설정을 주석 처리하고 다른 설정에 대한 주석 처리를 제거합니다.

In [ ]:
module_path = 'https://tfhub.dev/deepmind/bigbigan-resnet50/1'  # ResNet-50
# module_path = 'https://tfhub.dev/deepmind/bigbigan-revnet50x4/1'  # RevNet-50 x4

## 설정

In [ ]:
import io
import IPython.display
import PIL.Image
from pprint import pformat

import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow_hub as hub

## 이미지를 표시하는 일부 함수 정의하기

In [ ]:
def imgrid(imarray, cols=4, pad=1, padval=255, row_major=True):
  """Lays out a [N, H, W, C] image array as a single image grid."""
  pad = int(pad)
  if pad < 0:
    raise ValueError('pad must be non-negative')
  cols = int(cols)
  assert cols >= 1
  N, H, W, C = imarray.shape
  rows = N // cols + int(N % cols != 0)
  batch_pad = rows * cols - N
  assert batch_pad >= 0
  post_pad = [batch_pad, pad, pad, 0]
  pad_arg = [[0, p] for p in post_pad]
  imarray = np.pad(imarray, pad_arg, 'constant', constant_values=padval)
  H += pad
  W += pad
  grid = (imarray
          .reshape(rows, cols, H, W, C)
          .transpose(0, 2, 1, 3, 4)
          .reshape(rows*H, cols*W, C))
  if pad:
    grid = grid[:-pad, :-pad]
  return grid

def interleave(*args):
  """Interleaves input arrays of the same shape along the batch axis."""
  if not args:
    raise ValueError('At least one argument is required.')
  a0 = args[0]
  if any(a.shape != a0.shape for a in args):
    raise ValueError('All inputs must have the same shape.')
  if not a0.shape:
    raise ValueError('Inputs must have at least one axis.')
  out = np.transpose(args, [1, 0] + list(range(2, len(a0.shape) + 1)))
  out = out.reshape(-1, *a0.shape[1:])
  return out

def imshow(a, format='png', jpeg_fallback=True):
  """Displays an image in the given format."""
  a = a.astype(np.uint8)
  data = io.BytesIO()
  PIL.Image.fromarray(a).save(data, format)
  im_data = data.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print ('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format)
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def image_to_uint8(x):
  """Converts [-1, 1] float array to [0, 255] uint8."""
  x = np.asarray(x)
  x = (256. / 2.) * (x + 1.)
  x = np.clip(x, 0, 255)
  x = x.astype(np.uint8)
  return x

## BigBiGAN TF Hub 모듈을 로드하고 사용 가능한 기능 표시하기

In [ ]:
# module = hub.Module(module_path, trainable=True, tags={'train'})  # training
module = hub.Module(module_path)  # inference

for signature in module.get_signature_names():
  print('Signature:', signature)
  print('Inputs:', pformat(module.get_input_info_dict(signature)))
  print('Outputs:', pformat(module.get_output_info_dict(signature)))
  print()

## 다양한 함수에 편리하게 액세스할 수 있도록 래퍼 클래스 정의하기

In [ ]:
class BigBiGAN(object):

  def __init__(self, module):
    """Initialize a BigBiGAN from the given TF Hub module."""
    self._module = module

  def generate(self, z, upsample=False):
    """Run a batch of latents z through the generator to generate images.

    Args:
      z: A batch of 120D Gaussian latents, shape [N, 120].

    Returns: a batch of generated RGB images, shape [N, 128, 128, 3], range
      [-1, 1].
    """
    outputs = self._module(z, signature='generate', as_dict=True)
    return outputs['upsampled' if upsample else 'default']

  def make_generator_ph(self):
    """Creates a tf.placeholder with the dtype & shape of generator inputs."""
    info = self._module.get_input_info_dict('generate')['z']
    return tf.placeholder(dtype=info.dtype, shape=info.get_shape())

  def gen_pairs_for_disc(self, z):
    """Compute generator input pairs (G(z), z) for discriminator, given z.

    Args:
      z: A batch of latents (120D standard Gaussians), shape [N, 120].

    Returns: a tuple (G(z), z) of discriminator inputs.
    """
    # Downsample 256x256 image x for 128x128 discriminator input.
    x = self.generate(z)
    return x, z

  def encode(self, x, return_all_features=False):
    """Run a batch of images x through the encoder.

    Args:
      x: A batch of data (256x256 RGB images), shape [N, 256, 256, 3], range
        [-1, 1].
      return_all_features: If True, return all features computed by the encoder.
        Otherwise (default) just return a sample z_hat.

    Returns: the sample z_hat of shape [N, 120] (or a dict of all features if
      return_all_features).
    """
    outputs = self._module(x, signature='encode', as_dict=True)
    return outputs if return_all_features else outputs['z_sample']

  def make_encoder_ph(self):
    """Creates a tf.placeholder with the dtype & shape of encoder inputs."""
    info = self._module.get_input_info_dict('encode')['x']
    return tf.placeholder(dtype=info.dtype, shape=info.get_shape())

  def enc_pairs_for_disc(self, x):
    """Compute encoder input pairs (x, E(x)) for discriminator, given x.

    Args:
      x: A batch of data (256x256 RGB images), shape [N, 256, 256, 3], range
        [-1, 1].

    Returns: a tuple (downsample(x), E(x)) of discriminator inputs.
    """
    # Downsample 256x256 image x for 128x128 discriminator input.
    x_down = tf.nn.avg_pool(x, ksize=2, strides=2, padding='SAME')
    z = self.encode(x)
    return x_down, z

  def discriminate(self, x, z):
    """Compute the discriminator scores for pairs of data (x, z).

    (x, z) must be batches with the same leading batch dimension, and joint
      scores are computed on corresponding pairs x[i] and z[i].

    Args:
      x: A batch of data (128x128 RGB images), shape [N, 128, 128, 3], range
        [-1, 1].
      z: A batch of latents (120D standard Gaussians), shape [N, 120].

    Returns:
      A dict of scores:
        score_xz: the joint scores for the (x, z) pairs.
        score_x: the unary scores for x only.
        score_z: the unary scores for z only.
    """
    inputs = dict(x=x, z=z)
    return self._module(inputs, signature='discriminate', as_dict=True)

  def reconstruct_x(self, x, use_sample=True, upsample=False):
    """Compute BigBiGAN reconstructions of images x via G(E(x)).

    Args:
      x: A batch of data (256x256 RGB images), shape [N, 256, 256, 3], range
        [-1, 1].
      use_sample: takes a sample z_hat ~ E(x). Otherwise, deterministically
        use the mean. (Though a sample z_hat may be far from the mean z,
        typically the resulting recons G(z_hat) and G(z) are very
        similar.
      upsample: if set, upsample the reconstruction to the input resolution
        (256x256). Otherwise return the raw lower resolution generator output
        (128x128).

    Returns: a batch of recons G(E(x)), shape [N, 256, 256, 3] if
      `upsample`, otherwise [N, 128, 128, 3].
    """
    if use_sample:
      z = self.encode(x)
    else:
      z = self.encode(x, return_all_features=True)['z_mean']
    recons = self.generate(z, upsample=upsample)
    return recons

  def losses(self, x, z):
    """Compute per-module BigBiGAN losses given data & latent sample batches.

    Args:
      x: A batch of data (256x256 RGB images), shape [N, 256, 256, 3], range
        [-1, 1].
      z: A batch of latents (120D standard Gaussians), shape [M, 120].

    For the original BigBiGAN losses, pass batches of size N=M=2048, with z's
    sampled from a 120D standard Gaussian (e.g., np.random.randn(2048, 120)),
    and x's sampled from the ImageNet (ILSVRC2012) training set with the
    "ResNet-style" preprocessing from:

        https://github.com/tensorflow/tpu/blob/master/models/official/resnet/resnet_preprocessing.py

    Returns:
      A dict of per-module losses:
        disc: loss for the discriminator.
        enc: loss for the encoder.
        gen: loss for the generator.
    """
    # Compute discriminator scores on (x, E(x)) pairs.
    # Downsample 256x256 image x for 128x128 discriminator input.
    scores_enc_x_dict = self.discriminate(*self.enc_pairs_for_disc(x))
    scores_enc_x = tf.concat([scores_enc_x_dict['score_xz'],
                              scores_enc_x_dict['score_x'],
                              scores_enc_x_dict['score_z']], axis=0)

    # Compute discriminator scores on (G(z), z) pairs.
    scores_gen_z_dict = self.discriminate(*self.gen_pairs_for_disc(z))
    scores_gen_z = tf.concat([scores_gen_z_dict['score_xz'],
                              scores_gen_z_dict['score_x'],
                              scores_gen_z_dict['score_z']], axis=0)

    disc_loss_enc_x = tf.reduce_mean(tf.nn.relu(1. - scores_enc_x))
    disc_loss_gen_z = tf.reduce_mean(tf.nn.relu(1. + scores_gen_z))
    disc_loss = disc_loss_enc_x + disc_loss_gen_z

    enc_loss = tf.reduce_mean(scores_enc_x)
    gen_loss = tf.reduce_mean(-scores_gen_z)

    return dict(disc=disc_loss, enc=enc_loss, gen=gen_loss)

## 나중에 샘플, 재구성, 판별자 점수 및 손실 계산에 사용할 텐서 만들기

In [ ]:
bigbigan = BigBiGAN(module)

# Make input placeholders for x (`enc_ph`) and z (`gen_ph`).
enc_ph = bigbigan.make_encoder_ph()
gen_ph = bigbigan.make_generator_ph()

# Compute samples G(z) from encoder input z (`gen_ph`).
gen_samples = bigbigan.generate(gen_ph)

# Compute reconstructions G(E(x)) of encoder input x (`enc_ph`).
recon_x = bigbigan.reconstruct_x(enc_ph, upsample=True)

# Compute encoder features used for representation learning evaluations given
# encoder input x (`enc_ph`).
enc_features = bigbigan.encode(enc_ph, return_all_features=True)

# Compute discriminator scores for encoder pairs (x, E(x)) given x (`enc_ph`)
# and generator pairs (G(z), z) given z (`gen_ph`).
disc_scores_enc = bigbigan.discriminate(*bigbigan.enc_pairs_for_disc(enc_ph))
disc_scores_gen = bigbigan.discriminate(*bigbigan.gen_pairs_for_disc(gen_ph))

# Compute losses.
losses = bigbigan.losses(enc_ph, gen_ph)

## TensorFlow 세션을 만들고 변수 초기화하기

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 생성기 샘플

먼저 표준 가우스(`np.random.randn`을 통해)에서 생성기 입력 `z`를 샘플링하고 생성되는 이미지를 표시하여 사전 훈련된 BigBiGAN 생성기의 샘플을 시각화합니다. 아직까지는 표준 GAN의 기능을 넘어서는 것이 아닙니다. 지금은 단지 인코더를 무시하고 생성기를 사용합니다.

In [ ]:
feed_dict = {gen_ph: np.random.randn(32, 120)}
_out_samples = sess.run(gen_samples, feed_dict=feed_dict)
print('samples shape:', _out_samples.shape)
imshow(imgrid(image_to_uint8(_out_samples), cols=4))

# TF-Flowers 데이터세트에서 `test_images` 로드하기

BigBiGAN은 ImageNet에서 훈련되었지만 이 데모에서 작업하기에 너무 크기 때문에 재구성을 시각화하고 인코더 기능을 계산하기 위한 입력으로 더 작은 TF-Flowers [1] 데이터세트를 사용합니다.

이 셀에서 TF-Flowers를 로드하고(필요한 경우 데이터세트 다운로드) 256x256 RGB 이미지 샘플의 고정 배치를 NumPy 배열 `test_images`에 저장합니다.

[1] https://www.tensorflow.org/datasets/catalog/tf_flowers

In [ ]:
def get_flowers_data():
  """Returns a [32, 256, 256, 3] np.array of preprocessed TF-Flowers samples."""
  import tensorflow_datasets as tfds
  ds, info = tfds.load('tf_flowers', split='train', with_info=True)

  # Just get the images themselves as we don't need labels for this demo.
  ds = ds.map(lambda x: x['image'])

  # Filter out small images (with minor edge length <256).
  ds = ds.filter(lambda x: tf.reduce_min(tf.shape(x)[:2]) >= 256)

  # Take the center square crop of the image and resize to 256x256.
  def crop_and_resize(image):
    imsize = tf.shape(image)[:2]
    minor_edge = tf.reduce_min(imsize)
    start = (imsize - minor_edge) // 2
    stop = start + minor_edge
    cropped_image = image[start[0] : stop[0], start[1] : stop[1]]
    resized_image = tf.image.resize_bicubic([cropped_image], [256, 256])[0]
    return resized_image
  ds = ds.map(crop_and_resize)

  # Convert images from [0, 255] uint8 to [-1, 1] float32.
  ds = ds.map(lambda image: tf.cast(image, tf.float32) / (255. / 2.) - 1)

  # Take the first 32 samples.
  ds = ds.take(32)

  return np.array(list(tfds.as_numpy(ds)))

test_images = get_flowers_data()

# 재구성

이제 실제 이미지를 인코더를 통해 전달하고 생성기를 통해 다시 전달하여 주어진 `x` 이미지의 `G(E(x))`를 계산하는 식으로 BigBiGAN 재구성을 시각화합니다. 아래에서 입력 이미지 `x`는 왼쪽 열에 표시되고 해당 재구성은 오른쪽에 표시됩니다.

재구성은 입력 이미지를 픽셀 수준까지 완벽하게 일치시키는 작업이 아닙니다. 오히려 낮은 수준의 세부 사항을 대부분 "잊으면서" 입력에 대해 더 높은 수준의 의미론적 내용을 포착하는 데 중점을 둡니다. 이는 BigBiGAN 인코더가 표현 학습 접근 방식에서 우리가 보고자 하는 이미지에 대한 높은 수준의 의미론적 정보 유형을 포착하는 방법을 학습할 수 있음을 시사합니다.

또한 256x256 입력 이미지의 원시 재구성은 생성기에서 생성되는 더 낮은 해상도인 128x128에서 이루어진다는 점에 유의하세요. 시각화 목적으로 이 해상도를 업샘플링합니다.

In [ ]:
test_images_batch = test_images[:16]
_out_recons = sess.run(recon_x, feed_dict={enc_ph: test_images_batch})
print('reconstructions shape:', _out_recons.shape)

inputs_and_recons = interleave(test_images_batch, _out_recons)
print('inputs_and_recons shape:', inputs_and_recons.shape)
imshow(imgrid(image_to_uint8(inputs_and_recons), cols=2))

# 인코더 특성

이제 표준 표현 학습 평가에 사용되는 인코더에서 특성을 계산하는 방법을 시연합니다.

이러한 특성은 선형 또는 NN(nearest neighbor) 기반 분류자에서 사용할 수 있습니다. 최상의 결과를 얻기 위해 전역 평균 풀링 이후에 취해진 표준 특성(키 `avepool_feat`)과 더 큰 "BN + CReLU" 특성(키 `bn_crelu_feat`)을 모두 포함합니다.

In [ ]:
_out_features = sess.run(enc_features, feed_dict={enc_ph: test_images_batch})
print('AvePool features shape:', _out_features['avepool_feat'].shape)
print('BN+CReLU features shape:', _out_features['bn_crelu_feat'].shape)

# 판별자 점수 및 손실

마지막으로, 인코더 및 생성기 쌍의 배치에 대한 판별자 점수와 손실을 계산합니다. 이러한 손실은 BigBiGAN을 훈련하기 위해 옵티마이저로 전달될 수 있습니다.

위의 이미지 배치를 인코더 입력 `x`로 사용하여 인코더 점수를 `D(x, E(x))`로 계산합니다. 생성기 입력의 경우 `np.random.randn`을 통해 120D 표준 가우스에서 `z`를 샘플링하고 생성기 점수를 `D(G(z), z)`로 계산합니다.

판별자는 `(x, z)` 쌍에 대해 결합 점수 `score_xz`를 예측하고 `x` 및 `z` 각각에 대해 `score_x` 및 `score_z`의 단항 점수를 예측합니다. 인코더 쌍에 높은 (양의) 점수를 부여하고 생성기 쌍에 낮은 (음의) 점수를 부여하도록 훈련됩니다. 이것은 아래에서 단항 `score_z`가 두 경우 모두 음수이지만 인코더 출력 <code>E(x)</code>가 가우스의 실제 샘플과 유사하다는 것을 나타냅니다.

In [ ]:
feed_dict = {enc_ph: test_images, gen_ph: np.random.randn(32, 120)}
_out_scores_enc, _out_scores_gen, _out_losses = sess.run(
    [disc_scores_enc, disc_scores_gen, losses], feed_dict=feed_dict)
print('Encoder scores:', {k: v.mean() for k, v in _out_scores_enc.items()})
print('Generator scores:', {k: v.mean() for k, v in _out_scores_gen.items()})
print('Losses:', _out_losses)